In [3]:
import torch
import cv2
import os

class CustomModel(torch.nn.Module):
    def __init__(self, model_dict):
        super().__init__()
        self.model_dict = model_dict

    def forward(self, x):
        return self.model_dict(x)

# Load the pre-trained model from the .pt file
weights_path = 'saved_model.pt'
model_dict = torch.load(weights_path, map_location=torch.device('cpu'))['model'].float().state_dict()

# Create a custom module using the model dictionary
model = CustomModel(model_dict)

# Define the folder containing the images to be processed
image_folder = 'data/test_yolov5_1'



# Loop over each image in the folder and perform object detection
for image_file in os.listdir(image_folder):
    # Read the image file
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    # Convert the image to a PyTorch tensor
    image_tensor = torch.from_numpy(image).permute(2, 0, 1).float().div(255.0).unsqueeze(0)

    # Perform object detection using the YOLOv5 model
    with torch.no_grad():
        results = model(image_tensor)

    # Extract the coordinates of the bounding boxes of each object detected in the image
    bboxes = results.xyxy[0].cpu().numpy()

    # Loop over each bounding box and print the coordinates
    for bbox in bboxes:
        x1, y1, x2, y2, conf, cls = bbox.tolist()
        print(f"Object detected: class {int(cls)}, confidence {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}), ({x2:.2f}, {y2:.2f})")


KeyError: 'model'